In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
learning_rate = 0.001

X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.float32)
X_img = tf.reshape(X, [-1, 28, 28, 1])

with tf.name_scope("layer1"):
    # 28 x 28 -> 25 x 25
    w1 = tf.Variable(tf.get_variable("w1", shape=[4,4,1,32], initializer=tf.contrib.layers.xavier_initializer()))
    _conv1 = tf.nn.conv2d(X_img, w1, strides=[1,1,1,1], padding='VALID')
    conv1 = tf.nn.relu(_conv1)
    layer1 = tf.nn.max_pool(conv1, ksize=[1,2,2,1], strides=[1,1,1,1], padding='VALID')
    # layer1.shape = [n, 24, 24, 32]
    
with tf.name_scope("layer2"):
    # 24 x 24 -> 21 x 21
    w2 = tf.Variable(tf.get_variable("w2", shape=[4,4,32,64], initializer=tf.contrib.layers.xavier_initializer()))
    _conv2 = tf.nn.conv2d(layer1, w2, strides=[1,1,1,1], padding='VALID')
    conv2 = tf.nn.relu(_conv2)
    layer2 = tf.nn.max_pool(conv2, ksize=[1,4,4,1], strides=[1,1,1,1], padding='VALID')
    # 21 x 21 -> 18 x 18

with tf.name_scope("layer3"):
    w3 = tf.Variable(tf.get_variable("w3", shape=[18 * 18 * 64, 10], initializer=tf.contrib.layers.xavier_initializer()))
    b = tf.Variable(tf.zeros([10]))
    
with tf.name_scope("cost"):
    logit = tf.matmul(tf.reshape(layer2,[-1,18*18*64]), w3) + b
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logit, labels=Y))
    train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
    
    cost_hist = tf.summary.scalar("cost", cost)

with tf.name_scope("predict"):
    pred = tf.argmax(logit, 1)
    accuray = tf.reduce_mean(tf.cast(tf.equal(pred, tf.argmax(Y, 1)), tf.float32))
    
    accuray_hist = tf.summary.scalar("accuracy",accuray)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
with tf.Session() as sess:
    summary = tf.summary.merge_all()
    writer = tf.summary.FileWriter('./logs/mnist_problem/mnist_cnn')
    writer.add_graph(sess.graph)
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(5):
        batch_size = 100
        
        for step in range(mnist.train.num_examples / batch_size):
            x_data, y_data = mnist.train.next_batch(batch_size=batch_size)            
            sess.run(train, feed_dict={X:x_data, Y:y_data})
        x_test, y_test = mnist.test.next_batch(mnist.test.num_examples)
        s, accu = sess.run([summary, accuray], feed_dict={X:x_test, Y:y_test})
        writer.add_summary(s, global_step=str(epoch+1))
        
        print epoch+1, 'accuracy:', accu*100,'%'
    

1 accuracy: 98.55999946594238 %
2 accuracy: 98.9799976348877 %
3 accuracy: 99.33000206947327 %
4 accuracy: 99.11999702453613 %
5 accuracy: 99.23999905586243 %
